In [1]:
import numpy as np
import pandas as pd
# from scipy.io import loadmat
# from scipy.io import loadmat
import sys
from pathlib import Path
sys.path.append('..') 
import platform
import os
from pathlib import Path
import shutil
from utils.basics.data_org import curr_computer, move_subfolders
from pathlib import Path
import shutil
from utils.basics.data_org import curr_computer

In [9]:
def move_subfolders(dir1, dir2, subfolders=None):
    """
    Moves specified subfolders from dir1 to dir2 if they exist.
    If no subfolders are specified, moves all subfolders.

    :param dir1: Source directory
    :param dir2: Destination directory
    :param subfolders: List of subfolder names to move (optional)
    """
    # Ensure the destination directory exists
    os.makedirs(dir2, exist_ok=True)
    
    # Get the list of subfolders to move
    if subfolders is None:
        # Move all subfolders
        subfolders = [d for d in os.listdir(dir1) if os.path.isdir(os.path.join(dir1, d))]
    
    # Move each subfolder if it exists
    for subfolder in subfolders:
        src_path = os.path.join(dir1, subfolder)
        dest_path = os.path.join(dir2, subfolder)
        if os.path.exists(src_path) and os.path.isdir(src_path):
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
            print(f"Copied: {src_path} -> {dest_path}")
        else:
            print(f"Subfolder not found: {src_path}")

In [ ]:
animal_list = ['743986', '752703']
root = curr_computer()
target_root = r'C:\Users\svc_aind_behavior\Documents\sueData'
all_sessions_dir = []
for animal in animal_list:
    curr_sessions = [session for session in os.listdir(os.path.join(root, animal)) if os.path.isdir(os.path.join(root, animal, session))]
    for session in curr_sessions:
        curr_dir = os.path.join(root, animal, session)
        target_dir = os.path.join(target_root, animal, session)
        print(f"Current session: {session}")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
        move_subfolders(curr_dir, target_dir, subfolders=['behavior', 'pupil'])

Current session: m743986d20241203
Moved: E:\743986\m743986d20241203\behavior -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241203\behavior
Moved: E:\743986\m743986d20241203\pupil -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241203\pupil
Current session: m743986d20241204
Moved: E:\743986\m743986d20241204\behavior -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241204\behavior
Moved: E:\743986\m743986d20241204\pupil -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241204\pupil
Current session: m743986d20241206
Moved: E:\743986\m743986d20241206\behavior -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241206\behavior
Moved: E:\743986\m743986d20241206\pupil -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241206\pupil
Current session: m743986d20241209
Moved: E:\743986\m743986d20241209\behavior -> C:\Users\svc_aind_behavior\Documents\sueData\743986\m743986d20241209\behavior
Moved: E:\7439

In [11]:
move_subfolders(r'E:\752703\m752703d20241204', r'Z:\sueSu\752703\m752703d20241204', subfolders=['behavior', 'pupil'])


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Z:\\'

In [2]:
from utils.basics.data_org import curr_computer
from utils.behavior.session_utils import *
from utils.ephys.neuron_utils import *

In [4]:
def parse_session_string(file_or_folder, root):
    """
    Parses input string to generate corresponding pathData outputs.

    Args:
        file_or_folder (str): Session name or name of .asc file.
                              e.g., 'mBB041d20161006' or 'mBB041d20161006.asc'
        root (str): Root folder, e.g., 'G:\\'
        sep (str): Separator, e.g., '\\' or '/'

    Returns:
        dict: A dictionary containing session-related path data.
    """
    filename = file_or_folder
    # Split animal name and date
    animal_name, date = filename.split("d", 1)
    animal_name = animal_name[1:]  # Remove leading 'm'
    date = date[:9]
    session_folder = f"m{animal_name}{date}"

    if ".asc" in file_or_folder:  # Input is an .asc file
        behavioral_data_path = os.path.join(root, animal_name, session_folder, "behavior", filename)
        suptitle_name = filename.split(".asc")[0]
        save_fig_name = suptitle_name
        videopath = os.path.join(root, animal_name, session_folder, "pupil")
        sorted_folder_location = lick_path = None
    else:  # Input is the folder
        filepath = os.path.join(root, animal_name, session_folder, "behavior")
        all_files = list(Path(filepath).iterdir())
        file_ind = [
            file.name for file in all_files if file_or_folder + ".asc" in file.name
        ]

        if file_ind:
            behavioral_data_path = os.path.join(filepath, file_ind[0])
            suptitle_name = file_ind[0][:-4]  # Remove '.asc'
        else:  # If looking at a folder without behavioral data
            suptitle_name = None
            behavioral_data_path = None

        save_fig_name = suptitle_name
        videopath = os.path.join(root, animal_name, session_folder, "pupil")

        if file_or_folder[-1].isalpha():  # If last character is alphabetical
            sorted_folder_location = os.path.join(
                root, animal_name, session_folder, "sorted", f"session {file_or_folder[-1]}"
            )
            lick_path = os.path.join(root, animal_name, session_folder, "lick", file_or_folder[-1])
        else:
            sorted_folder_location = os.path.join(root, animal_name, session_folder, "sorted", "session")
            lick_path = os.path.join(root, animal_name, session_folder, "lick", "session")

    # Construct output dictionary
    path_data = {
        "aniName": animal_name,
        "suptitleName": suptitle_name,
        "sessionFolder": session_folder,
        "sortedFolder": sorted_folder_location,
        "animalName": animal_name,
        "saveFigName": save_fig_name,
        "saveFigFolder": os.path.join(root, animal_name, session_folder, "figures"),
        "baseFolder": os.path.join(root, animal_name, session_folder),
        "behavioralDataPath": behavioral_data_path,
        "date": date,
        "videopath": videopath,
        "lickPath": lick_path,
    }

    # Check for neuralynx folders
    base_folder = path_data["baseFolder"]
    nlynx_folder = Path(base_folder, "neuralynx")
    if nlynx_folder.is_dir():
        path_data.update(
            {
                "nLynxFolder": f"{base_folder}neuralynx{sep}",
                "nLynxFolderOpto": f"{base_folder}neuralynx{sep}opto{sep}",
                "nLynxFolderSession": f"{base_folder}neuralynx{sep}session{sep}",
            }
        )

    return path_data


In [ ]:
session = 'mZS061d20210328'
root = curr_computer()
dir = parse_session_string(session, root)

In [15]:
animal_list = ['749624', '749472', '754895', '754896', '754898']
root = curr_computer()
target_root = r'F:\temporal_transfer'
all_sessions_dir = []
for animal in animal_list:
    curr_sessions = [session for session in os.listdir(os.path.join(root, animal)) if os.path.isdir(os.path.join(root, animal, session))]
    for session in curr_sessions:
        curr_dir = os.path.join(root, animal, session)
        target_dir = os.path.join(target_root, animal, session)
        print(f"Current session: {session}")
        if not os.path.exists(target_dir):
            os.makedirs(target_dir)
        move_subfolders(curr_dir, target_dir, subfolders=['behavior', 'pupil'])

Current session: m749624d20250101
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250101\behavior
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250101\pupil
Current session: m749624d20250102
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250102\behavior
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250102\pupil
Current session: m749624d20250103
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250103\behavior
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250103\pupil
Current session: m749624d20250104
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250104\behavior
Subfolder already exists: F:\temporal_transfer\749624\m749624d20250104\pupil
Current session: m749624d20250105
Subfolder not found: C:\Users\svc_aind_behavior\Documents\sueData\749624\m749624d20250105\behavior
Subfolder not found: C:\Users\svc_aind_behavior\Documents\sueData\749624\m749624d20250105\pupil
Current